In [2]:
# library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans

In [4]:
# import data
df = pd.read_csv('../data/tellco_data.csv')

In [ ]:
class DataInfo:
    def __init__(self, df: pd.DataFrame):
        self.df = df

         def fix_outlier_columns(self, columns: list):
        try:
            for column in columns:
                self.df[column] = np.where(self.df[column] > self.df[column].quantile(
                    0.95), self.df[column].median(), self.df[column])
        except:
            print("Cant fix outliers for each column")

        return self.df
    
    
def get_mode(self):
        mode = self.df.mode()
        actual_mode = mode.iloc[0, :]
        actual_mode.name = 'Mode'
        return actual_mode
    
def get_min_max_of_column(self, col, range=1):
        sortedVal = np.sort(self.df[col].unique())
        top_df = pd.DataFrame(sortedVal[::-1][:range], columns=['Max Value/s'])
        bottom_df = pd.DataFrame(sortedVal[:range], columns=['Min Value/s'])
        info_df = pd.concat([top_df, bottom_df], axis=1)
        return info_df
    
def standardize_column(self, column: str) -> pd.DataFrame:
        try:
            std_column_df = pd.DataFrame(self.df[column])
            std_column_values = std_column_df.values
            standardizer = StandardScaler()
            normalized_data = standardizer.fit_transform(std_column_values)
            self.df[column] = normalized_data

            return self.df
        except:
            print("Failed to standardize the column")